# ATUS Group Populations Project 
CS765

Rebecca Roembke & Katherine Heidi Fehr


## Resources:
- Helpful Pandas data encoding: https://pbpython.com/categorical-encoding.html
- Altair color scales: https://vega.github.io/vega/docs/schemes/

## Helpful Notes
**How to make variable categorical**
```python
# Convert column to categorical data:
data["TESEX"] = data["TESEX"].astype('category')
````


## Set Up

In [15]:
import pandas as pd
import altair as alt
import numpy as np

In [16]:
# NEED the below line in order to use such a large data set, 
# instead of storing the data in the jupyter notebook, 
# it leaves the file hosted locally
# More info: https://altair-viz.github.io/user_guide/faq.html
alt.data_transformers.enable('json')

# Import Data
data=pd.read_csv("Data/atussum_1121-reduced.csv")

# Set selection
pts = alt.selection(type="single", encodings=['x'])

## Data Clean-up
tesex_map = {1: "Male", 2: "Female"}
data["TESEX"] = data["TESEX"].map(tesex_map)

# AI/AN = American Indian/Alaska Native 
# NHOPI = Native Hawaiian or Other Pacific Islander
race_map = {-3:"Unknown", -2: "Unknown", -1:"Unknown",1: "White Only", 2: "Black Only", 3: "AI/AN Only", 4: "Asian Only",5: "NHOPI Only", 6:"> 1 race",7:"> 1 race",8:"> 1 race",9:"> 1 race",10:"> 1 race",11:"> 1 race",12:"> 1 race",13:"> 1 race",14:"> 1 race",15:"> 1 race",16:"> 1 race",17:"> 1 race",18:"> 1 race",19:"> 1 race",20:"> 1 race",21:"> 1 race",22:"> 1 race",23:"> 1 race",24:"> 1 race",25:"> 1 race"}
data["PTDTRACE"] = data["PTDTRACE"].map(race_map)

telfs_map = {1: "Employed", 2: "Employed", 3: "Unemployed", 4: "Unemployed",5: "Not in Labor Force"}
data["TELFS"] = data["TELFS"].map(telfs_map)

data["TRERNWA"] = data["TRERNWA"].replace(-1,np.NaN)
data["TRERNWA"] = data["TRERNWA"]/100

# Show sample
data.head()


,TUCASEID,GEMETSTA,GTMETSTA,PEEDUCA,PEHSPNON,PTDTRACE,TEAGE,TELFS,TEMJOT,TESCHENR,...,t09,t10,t11,t12,t13,t14,t15,t16,t18,t50
0,20110101110010,-1,1,37,2,Black Only,62,Not in Labor Force,-1,-1,...,0,0,10,688,0,0,0,0,12,0
1,20110101110072,-1,1,39,2,White Only,22,Employed,2,2,...,0,0,95,0,0,0,0,0,25,0
2,20110101110074,-1,2,36,2,White Only,33,Employed,2,2,...,0,0,25,180,0,0,0,0,10,525
3,20110101110081,-1,1,39,1,White Only,45,Employed,2,2,...,0,0,30,868,0,0,0,0,0,0
4,20110101110082,-1,1,39,2,White Only,24,Employed,2,2,...,0,0,95,70,0,0,0,0,40,0


## Manual Binning
To avoid unexpected artifacts in our figures, we need to manually bin some categories.

In [17]:
#### Age
data.loc[data['TEAGE'].between(0,15,'both'), 'AGE'] = '0-15'
data.loc[data['TEAGE'].between(15,20,'right'), 'AGE'] = '16-20'

cAge = 15
while cAge < 81:
    myString = str(cAge+1)+"-"+str(cAge+5)
    data.loc[data['TEAGE'].between(cAge,cAge+5,'right'), 'AGE'] = myString #right side inclusive
    cAge = cAge + 5
data.loc[data['TEAGE'].between(85,100,'right'), 'AGE'] = '85+'

#### Weekly income 'TRERNWA'
# Start list of strings in order
orderIncome = ['Unknown','$0-$200']
# Make first string
data.loc[data['TRERNWA'].between(0,200,'both'), 'WINCOME'] = '$0-$200'

sIncome = 200
increment = 200
while sIncome < 2801:
    myString = "$"+str(sIncome+1)+"-"+"$"+str(sIncome+increment)
    data.loc[data['TRERNWA'].between(sIncome,sIncome+increment,'right'), 'WINCOME'] = myString #right side inclusive
    orderIncome.append(myString)
    sIncome = sIncome + increment
data.loc[data['TRERNWA'].between(3001,6000,'right'), 'WINCOME'] = '$3000+'
orderIncome.append('$3000+')

data['WINCOME'] = data['WINCOME'].replace(np.NaN, 'Unknown')
# orderIncome.append('Unknown')


## Chart Generation

### Create Selector

In [18]:
pts = alt.selection_interval(
    encodings=['x'],
    resolve='intersect',
    empty="all",
    mark=alt.BrushConfig(fill='green')
)

### Constants

In [19]:
## Yaxis
yWork = [0, 1200]
yPC = [0, 1400]
yLeisure = [0, 1400]

myOP = alt.value(0.9)
circColor = 'lightgrey'
barColor='darkslateblue'
circRange = [0,100]

widthBarTotal = 1200


### Tooltip Definitions

In [20]:
tSex = alt.Tooltip('TESEX', title="Sex")
tEmploy = alt.Tooltip('TELFS', title="Employment Status")
tRace = alt.Tooltip('PTDTRACE', title="Race")

tAge = alt.Tooltip('AGE:N', title="Age")
tInc = alt.Tooltip('WINCOME', title="Weekly Income")
tChiNum = alt.Tooltip('TRCHILDNUM', title="Number of Children")

tPC = alt.Tooltip('t01:Q', title="Personal Care (minutes spent)")
tWork = alt.Tooltip('t05:Q', title="Work (minutes spent)")
tLeisure = alt.Tooltip('t12:Q', title="Leisure (minutes spent)")

### Bar Chart Filter Charts

In [21]:
# Make basic M/F bar chart
MF_bar = alt.Chart(data).mark_bar().encode(
    alt.X(
        "TESEX", 
        title=""
        ),
    alt.Y(
        'count()'
    ),
    color=alt.ColorValue("darkslateblue"),
    tooltip=[tSex, 'count()']
).add_selection(pts
).properties(
    width=120,
    height=150,
    title='Sex'
)

#make employment bar chart
Employment_bar=alt.Chart(data).mark_bar().encode(
    alt.X(
        "TELFS", 
        title=""),
    alt.Y(
        'count()'
    ),
    color=alt.ColorValue("darkslateblue"),
    tooltip=[tEmploy, 'count()']
).add_selection(pts
).properties(
    width=180,
    height=150,
    title="Employment Status"
)

#make race bar chart
Race_bar=alt.Chart(data).mark_bar().encode(
    alt.X (
        "PTDTRACE",
        title="",
        sort=alt.EncodingSortField(field="PTDTRACE", op="count", order='descending')
    ),
    alt.Y('count()'),
    color=alt.ColorValue("darkslateblue"),
    tooltip=[tRace, 'count()']
).add_selection(pts
).properties(
    width=350,
    height=150,
    title="Race"
)



(MF_bar|Employment_bar|Race_bar)



alt.HConcatChart(...)

### A: Age

In [22]:
colorAvg_work = alt.Chart(data).mark_rect(clip=True).encode(
    alt.X(
        'AGE:N',
        title="",
        axis=alt.Axis(labels=False)
        ),
    alt.Y(
        't05:Q',bin=alt.Bin(maxbins=20),
        scale=alt.Scale(domain=yWork),
        title = "Work"),
    alt.Color('count()',
        scale=alt.Scale(type='log',scheme='lighttealblue',domain=[10, 1500]),
        # scale=alt.Scale(scheme='blues',domain=[0, 10, 300, 1500], range=["white", "white", "pink", "red"]),
        legend=alt.Legend(title='Total Records')
    ),
    # opacity = myOP
).properties(
    width=200,
    height=150
)

circ_work = colorAvg_work.mark_point(clip=True).encode(
    alt.ColorValue(circColor),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection'),
        scale = alt.Scale(domain=[10,1500],range=circRange)
    ),
    tooltip=[tAge, 'count()']
).transform_filter(
    pts
).interactive(bind_y=False
) 

colorAvg_PC = alt.Chart(data).mark_rect(clip=True).encode(
    alt.X(
        'AGE:N',
        title="",
        axis=alt.Axis(labels=False)
        ),
    alt.Y(
        't01:Q',bin=alt.Bin(maxbins=20),
        scale=alt.Scale(domain=yPC),
        title = "Personal Care"),
    alt.Color('count()',
        scale=alt.Scale(type='log',scheme='lighttealblue',domain=[10, 1500]),
        # scale=alt.Scale(scheme='blues',domain=[0, 10, 300, 1500], range=["white", "white", "pink", "red"]),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=200,
    height=150
)

circ_PC = colorAvg_PC.mark_point(clip=True).encode(
    alt.ColorValue(circColor),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection'),
        scale = alt.Scale(domain=[10,1500],range=circRange)
    ),
    tooltip=[tAge, 'count()']
).transform_filter(
    pts
).interactive(bind_y=False) 

colorAvg_Leisure = alt.Chart(data).mark_rect(clip=True).encode(
    alt.X(
        'AGE:N',
        title="Age"
        ),
    alt.Y(
        't12:Q',bin=alt.Bin(maxbins=20),
        scale=alt.Scale(domain=yLeisure),
        title="Leisure"),
    alt.Color('count()',
        scale=alt.Scale(type='log',scheme='lighttealblue',domain=[10, 1500]),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=200,
    height=150
)

circ_Leisure = colorAvg_Leisure.mark_point(clip=True).encode(
    alt.ColorValue(circColor),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection'),
        scale = alt.Scale(domain=[10,1500],range=circRange)
    ),
    tooltip=[tAge, 'count()']
).transform_filter(
    pts
).interactive(bind_y=False) 

alt.vconcat(
    (MF_bar|Employment_bar|Race_bar),
    colorAvg_work+circ_work,
    colorAvg_PC+circ_PC,
    colorAvg_Leisure+circ_Leisure,
).resolve_legend(
    color="independent",
    size="independent"
).configure_concat(
    spacing=0
)

alt.VConcatChart(...)

### A: Weekly Earnings

In [23]:
# TRERNWA 

colorAvg_weekEarn_work = alt.Chart(data).mark_rect(clip=True).encode(
    alt.X(
        'WINCOME', 
        sort=orderIncome,
        title="",
        axis=alt.Axis(labels=False)
        # bin=alt.Bin(maxbins=16),
        # scale= alt.Scale(domain=[0,3000])
        ),
    alt.Y(
        't05:Q',bin=alt.Bin(maxbins=20),
        scale=alt.Scale(domain=yWork),
        title="",
        # title = "Work"
        ),
    alt.Color('count()',
        scale=alt.Scale(type='log',scheme='lighttealblue',domain=[10, 1500]),
        # scale=alt.Scale(scheme='blues',domain=[0, 10, 300, 1500], range=["white", "white", "pink", "red"]),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=200,
    height=150
)

circ_weekEarn_work = colorAvg_weekEarn_work.mark_point(clip=True).encode(
    alt.ColorValue(circColor),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection'),
        scale = alt.Scale(domain=[10,1500],range=circRange)
    ),
    tooltip=[tInc, 'count()']
).transform_filter(
    pts
).interactive(bind_y=False)

colorAvg_weekEarn_PC = alt.Chart(data).mark_rect(clip=True).encode(
    alt.X(
        'WINCOME',
        sort=orderIncome,
        title="",
        axis=alt.Axis(labels=False)
        # bin=alt.Bin(maxbins=16),
        # scale= alt.Scale(domain=[0,3000])
        ),
    alt.Y(
        't01:Q',bin=alt.Bin(maxbins=20),
        scale=alt.Scale(domain=yPC),
        title="",
        # title = "Personal Care"
        ),
    alt.Color('count()',
        scale=alt.Scale(type='log',scheme='lighttealblue',domain=[10, 1500]),
        # scale=alt.Scale(scheme='blues',domain=[0, 10, 300, 1500], range=["white", "white", "pink", "red"]),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=200,
    height=150
)

circ_weekEarn_PC = colorAvg_weekEarn_PC.mark_point(clip=True).encode(
    alt.ColorValue(circColor),
    alt.Size('count()',
        # bin=True,
        legend=alt.Legend(title='Records in Selection'),
        scale = alt.Scale(domain=[10,1500],range=circRange)
    ),
    tooltip=[tInc, 'count()']
).transform_filter(
    pts
).interactive(bind_y=False)

colorAvg_weekEarn_Leisure = alt.Chart(data).mark_rect(clip=True).encode(
    alt.X(
        'WINCOME', 
        sort=orderIncome,
        title="Weekly Income", 
        # bin=alt.Bin(maxbins=16),
        # scale= alt.Scale(domain=[0,3000])
        ),
    alt.Y(
        't12:Q',bin=alt.Bin(maxbins=20),
        scale=alt.Scale(domain=yLeisure),
        title="",
        # title="Leisure"
        ),
    alt.Color('count()',
        scale=alt.Scale(type='log',scheme='lighttealblue',domain=[10, 1500]),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=200,
    height=150
)

circ_weekEarn_Leisure = colorAvg_weekEarn_Leisure.mark_point(clip=True).encode(
    alt.ColorValue(circColor),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection'),
        scale = alt.Scale(domain=[10,1500],range=circRange)
    ),
    tooltip=[tInc, 'count()']
).transform_filter(
    pts
).interactive(bind_y=False)

alt.vconcat(
    (MF_bar|Employment_bar|Race_bar),
    colorAvg_weekEarn_work+circ_weekEarn_work | colorAvg_work+circ_work,
    colorAvg_weekEarn_PC+circ_weekEarn_PC| colorAvg_PC+circ_PC,
    colorAvg_weekEarn_Leisure+circ_weekEarn_Leisure | colorAvg_Leisure+circ_Leisure,

).resolve_legend(
    color="independent",
    size="independent"
)




alt.VConcatChart(...)

### A: Number of Children

In [24]:
# TRCHILDNUM

colorAvg_childNum_work = alt.Chart(data).mark_rect(clip=True).encode(
    alt.X(
        'TRCHILDNUM:N', 
        title="",
        axis=alt.Axis(labels=False),
        # scale= alt.Scale(domain=[0,10])
        ),
    alt.Y(
        't05:Q',bin=alt.Bin(maxbins=20),
        scale=alt.Scale(domain=yWork),
        title="",
        # title = "Work"
        ),
    alt.Color('count()',
        scale=alt.Scale(type='log',scheme='lighttealblue',domain=[10, 1500]),
        # scale=alt.Scale(scheme='blues',domain=[0, 10, 300, 1500], range=["white", "white", "pink", "red"]),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=200,
    height=150
)

circ_childNum_work = colorAvg_childNum_work.mark_point(clip=True).encode(
    alt.ColorValue(circColor),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection'),
        scale = alt.Scale(domain=[10,1500],range=circRange)
    ),
    tooltip=[tChiNum, 'count()']
).transform_filter(
    pts
).interactive(bind_y=False)

colorAvg_childNum_PC = alt.Chart(data).mark_rect(clip=True).encode(
    alt.X(
        'TRCHILDNUM:N', 
        title="",
        axis=alt.Axis(labels=False)
        # scale= alt.Scale(domain=[0,3000])
        ),
    alt.Y(
        't01:Q',bin=alt.Bin(maxbins=20),
        scale=alt.Scale(domain=yPC),
        title="",
        # title = "Personal Care"
        ),
    alt.Color('count()',
        scale=alt.Scale(type='log',scheme='lighttealblue',domain=[10, 1500]),
        # scale=alt.Scale(scheme='blues',domain=[0, 10, 300, 1500], range=["white", "white", "pink", "red"]),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=200,
    height=150
)

circ_childNum_PC = colorAvg_childNum_PC.mark_point(clip=True).encode(
    alt.ColorValue(circColor),
    alt.Size('count()',
        # bin=True,
        legend=alt.Legend(title='Records in Selection'),
        scale = alt.Scale(domain=[10,1500],range=circRange)
    ),
    tooltip=[tChiNum, 'count()']
).transform_filter(
    pts
).interactive(bind_y=False)

colorAvg_childNum_Leisure = alt.Chart(data).mark_rect(clip=True).encode(
    alt.X(
        'TRCHILDNUM:N',
        title="Number of Children", 
        ),
    alt.Y(
        't12:Q',bin=alt.Bin(maxbins=20),
        scale=alt.Scale(domain=yLeisure),
        title="",
        # title="Leisure"
        ),
    alt.Color('count()',
        scale=alt.Scale(type='log',scheme='lighttealblue',domain=[10, 1500]),
        legend=alt.Legend(title='Total Records')
    )
).properties(
    width=200,
    height=150
)

circ_childNum_Leisure = colorAvg_childNum_Leisure.mark_point(clip=True).encode(
    alt.ColorValue(circColor),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection'),
        scale = alt.Scale(domain=[10,1500],range=circRange)
    ),
    tooltip=[tChiNum, 'count()']
).transform_filter(
    pts
).interactive(bind_y=False)

out = alt.vconcat(
    (MF_bar|Employment_bar|Race_bar),
    colorAvg_work+circ_work | colorAvg_childNum_work+circ_childNum_work| colorAvg_weekEarn_work+circ_weekEarn_work,
    colorAvg_PC+circ_PC | colorAvg_childNum_PC+circ_childNum_PC | colorAvg_weekEarn_PC+circ_weekEarn_PC,
    colorAvg_Leisure+circ_Leisure | colorAvg_childNum_Leisure+circ_childNum_Leisure | colorAvg_weekEarn_Leisure+circ_weekEarn_Leisure,

).resolve_legend(
    color="independent",
    size="independent"
).configure_concat(
    spacing=15
)

out

alt.VConcatChart(...)

### Histograms

In [25]:
#HISTOGRAMS
base=alt.Chart(data)

bar_args = {'opacity': .3, 'binSpacing': 0}


#age histograms
top_Age_hist_interactive = colorAvg_work.mark_bar(clip=True).encode(
    alt.X('AGE:N',
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.Y('count()', stack=None, title='Records'),
    tooltip=[tAge, 'count()'],
    color=alt.ColorValue(barColor)
).properties(height=59, width=200
).transform_filter(
    pts
)

top_Age_hist = base.mark_bar(clip=True).encode(
    alt.X('AGE:N',
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.Y('count()', stack=None, title='Records'),
    tooltip=[tAge, 'count()'],
    color=alt.ColorValue(circColor)
).properties(height=59, width=200
)

#number of kids histograms
top_kids_hist_interactive = colorAvg_childNum_Leisure.mark_bar(clip=True).encode(
    alt.X('TRCHILDNUM:N',
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.Y('count()', stack=None, title=''),
    tooltip=[tChiNum, 'count()'],
    color=alt.ColorValue(barColor)
).properties(height=60, width=200
).transform_filter(
    pts
)


top_kids_hist = base.mark_bar(clip=True).encode(
    alt.X('TRCHILDNUM:N',
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.Y('count()', stack=None, title=''),
    tooltip=[tChiNum, 'count()'],
    color=alt.ColorValue(circColor)
).properties(height=60, width=200
)

#weekly income histograms
top_income_hist_interactive = colorAvg_weekEarn_work.mark_bar(clip=True).encode(
    alt.X('WINCOME',
        sort=orderIncome,
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.Y('count()', stack=None, title=''),
    tooltip=[tInc, 'count()'],
    color=alt.ColorValue(barColor)
).properties(height=60, width=200
).transform_filter(
    pts
)

top_income_hist = base.mark_bar(clip=True).encode(
    alt.X('WINCOME',
        sort=orderIncome,
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.Y('count()', stack=None, title=''),
    tooltip=[tInc, 'count()'],
    color=alt.ColorValue(circColor)
).properties(height=60, width=200
)

#work histograms

right_work_hist_interactive = colorAvg_work.mark_bar(clip=True).encode(
    alt.Y('t05:Q',
        bin=alt.Bin(maxbins=20),
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.X('count()', stack=None, title=''),
    tooltip=[tWork, 'count()'],
    color=alt.ColorValue(barColor)
).properties(width=60, height=150
).transform_filter(
    pts
)

right_work_hist = base.mark_bar(clip=True).encode(
    alt.Y('t05:Q',
          bin=alt.Bin(maxbins=20),
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.X('count()', stack=None, title=''),
    tooltip=[tWork, 'count()'],
    color=alt.ColorValue(circColor)
).properties(width=60, height=150
) 

#personal care histograms

right_PC_hist_interactive = colorAvg_PC.mark_bar(clip=True).encode(
    alt.Y('t01:Q',
        bin=alt.Bin(maxbins=20),
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.X('count()', stack=None, title=''),
    tooltip=[tPC, 'count()'],
    color=alt.ColorValue(barColor)
).properties(width=60, height=150
).transform_filter(
    pts
)

right_PC_hist = base.mark_bar(clip=True).encode(
    alt.Y('t01:Q',
          bin=alt.Bin(maxbins=20),
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.X('count()', stack=None, title=''),
    tooltip=[tPC, 'count()'],
    color=alt.ColorValue(circColor)
).properties(width=60, height=150
)   

#Leisure histograms

right_Leisure_hist_interactive = colorAvg_Leisure.mark_bar(clip=True).encode(
    alt.Y('t12:Q',
        bin=alt.Bin(maxbins=20),
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.X('count()', stack=None, title='Records'),
    tooltip=[tLeisure, 'count()'],
    color=alt.ColorValue(barColor)
).properties(width=60, height=150
).transform_filter(
    pts
)

right_Leisure_hist = base.mark_bar(clip=True).encode(
    alt.Y('t12:Q',
          bin=alt.Bin(maxbins=20),
          stack=None,
          title='',
          axis=alt.Axis(labels=False)
         ),
    alt.X('count()', stack=None, title='Records'),
    tooltip=[tLeisure, 'count()'],
    color=alt.ColorValue(circColor)
).properties(width=60, height=150
)


out = alt.vconcat(
    (MF_bar|Employment_bar|Race_bar),
    top_Age_hist+top_Age_hist_interactive|top_kids_hist+top_kids_hist_interactive|top_income_hist+top_income_hist_interactive,
    colorAvg_work+circ_work |colorAvg_childNum_work+circ_childNum_work| colorAvg_weekEarn_work+circ_weekEarn_work|right_work_hist+right_work_hist_interactive,
    colorAvg_PC+circ_PC | colorAvg_childNum_PC+circ_childNum_PC | colorAvg_weekEarn_PC+circ_weekEarn_PC|right_PC_hist+right_PC_hist_interactive,
    colorAvg_Leisure+circ_Leisure|colorAvg_childNum_Leisure+circ_childNum_Leisure | colorAvg_weekEarn_Leisure+circ_weekEarn_Leisure|right_Leisure_hist+right_Leisure_hist_interactive,

).resolve_legend(
    color="independent",
    size="independent"
).configure_concat(
    spacing=15
)

out

alt.VConcatChart(...)

## Export chart

In [26]:
with alt.data_transformers.enable('default'):
    out.save('chart.html')

with alt.data_transformers.enable('default'):
    out.save('ATUS_All.json')